
# Neuron - Classificação de Emoções em Português

Este notebook implementa o pipeline completo de **Machine Learning** para o projeto Neuron:

1. Carregamento do dataset de frases em português.
2. Análise exploratória simples.
3. Pré-processamento de texto (PT-BR).
4. Criação de dois modelos de IA:
   - **Modelo 1**: classificação de **emoções** (`alegria, tristeza, raiva, medo, amor, surpresa`).
   - **Modelo 2**: classificação de **sentimento** (`positivo` x `negativo`).
5. Salvamento dos modelos em arquivo `.pkl`.
6. Exemplo de uso do modelo para rotular uma conversa ao longo de vários dias e gerar gráficos.


In [ ]:

import pandas as pd
import numpy as np
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Executar apenas na primeira vez no ambiente
# nltk.download('stopwords')
# nltk.download('rslp')


## 1. Carregar dados

In [ ]:

# Ajuste o caminho se precisar
df = pd.read_csv("dados_humor_neuron_pt.csv")

df.head()


## 2. Análise exploratória simples

In [ ]:
df['emocao'].value_counts()

## 3. Criar coluna de sentimento (positivo/negativo)

In [ ]:

map_sent = {
    "alegria": "positivo",
    "amor": "positivo",
    "surpresa": "positivo",
    "tristeza": "negativo",
    "raiva": "negativo",
    "medo": "negativo",
}

df["sentimento"] = df["emocao"].map(map_sent)
df.head()


## 4. Pré-processamento de texto em PT-BR

In [ ]:

stopwords_pt = set(stopwords.words("portuguese"))
stemmer = RSLPStemmer()

def preprocess_pt(texto: str) -> str:
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúâêôãõç\s]", " ", texto)
    tokens = texto.split()
    tokens = [t for t in tokens if t not in stopwords_pt]
    tokens = [stemmer.stem(t) for t in tokens]
    return " ".join(tokens)

df["texto_limpo"] = df["texto"].apply(preprocess_pt)
df[["texto", "texto_limpo", "emocao", "sentimento"]].head()


## 5. Separar treino e teste

In [ ]:

X_train, X_test, y_emocao_train, y_emocao_test = train_test_split(
    df["texto_limpo"], df["emocao"],
    test_size=0.2, random_state=42, stratify=df["emocao"]
)

_, _, y_sent_train, y_sent_test = train_test_split(
    df["texto_limpo"], df["sentimento"],
    test_size=0.2, random_state=42, stratify=df["sentimento"]
)

len(X_train), len(X_test)


## 6. Vetorização TF-IDF

In [ ]:

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

X_train_vec.shape, X_test_vec.shape


## 7. Modelo 1 – Classificação de Emoções

In [ ]:

modelo_emocao = LogisticRegression(max_iter=1000, multi_class="multinomial")
modelo_emocao.fit(X_train_vec, y_emocao_train)

y_pred_emocao = modelo_emocao.predict(X_test_vec)
print("Accuracy EMOÇÃO:", accuracy_score(y_emocao_test, y_pred_emocao))
print(classification_report(y_emocao_test, y_pred_emocao))


## 8. Modelo 2 – Classificação de Sentimento (positivo/negativo)

In [ ]:

modelo_sentimento = LogisticRegression(max_iter=1000)
modelo_sentimento.fit(X_train_vec, y_sent_train)

y_pred_sent = modelo_sentimento.predict(X_test_vec)
print("Accuracy SENTIMENTO:", accuracy_score(y_sent_test, y_pred_sent))
print(classification_report(y_sent_test, y_pred_sent))


## 9. Salvar modelos em arquivo `.pkl`

In [ ]:

artefatos = {
    "vectorizer": vectorizer,
    "modelo_emocao": modelo_emocao,
    "modelo_sentimento": modelo_sentimento,
}

with open("modelos_neuron_pt.pkl", "wb") as f:
    pickle.dump(artefatos, f)

print("Modelos salvos em 'modelos_neuron_pt.pkl'")


## 10. Exemplo – Classificar uma conversa e gerar análise por dia

In [ ]:

# Carrega modelos (caso esteja em outro ambiente)
with open("modelos_neuron_pt.pkl", "rb") as f:
    artefatos2 = pickle.load(f)

vectorizer2 = artefatos2["vectorizer"]
modelo_emocao2 = artefatos2["modelo_emocao"]
modelo_sent2 = artefatos2["modelo_sentimento"]

# Exemplo de conversa com timestamps
dados_conversa = [
    {"timestamp": "2025-11-14 09:10:00", "texto": "Bom dia, estou bem cansado e desmotivado hoje."},
    {"timestamp": "2025-11-14 11:30:00", "texto": "A reunião foi ótima, estou animado com o resultado."},
    {"timestamp": "2025-11-15 10:05:00", "texto": "Estou preocupado com o prazo, com medo de atrasar."},
    {"timestamp": "2025-11-15 16:40:00", "texto": "Adorei o feedback do cliente, fiquei muito feliz."},
]

df_msgs = pd.DataFrame(dados_conversa)
df_msgs["timestamp"] = pd.to_datetime(df_msgs["timestamp"])
df_msgs["data"] = df_msgs["timestamp"].dt.date

df_msgs["texto_limpo"] = df_msgs["texto"].apply(preprocess_pt)
X_msgs = vectorizer2.transform(df_msgs["texto_limpo"])

df_msgs["emocao"] = modelo_emocao2.predict(X_msgs)
df_msgs["sentimento"] = modelo_sent2.predict(X_msgs)

df_msgs


### Gráficos de emoções e sentimento por dia

In [ ]:

emocoes_por_dia = (
    df_msgs.groupby(["data", "emocao"])
           .size()
           .reset_index(name="quantidade")
)

plt.figure(figsize=(8,4))
sns.lineplot(data=emocoes_por_dia, x="data", y="quantidade", hue="emocao", marker="o")
plt.title("Emoções por dia")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

sent_por_dia = (
    df_msgs.groupby(["data", "sentimento"])
           .size()
           .reset_index(name="quantidade")
)

plt.figure(figsize=(8,4))
sns.barplot(data=sent_por_dia, x="data", y="quantidade", hue="sentimento")
plt.title("Sentimento positivo/negativo por dia")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
